<h1>kNN model on kaggle-pet competition</h1>

we are following Stanford CS231n, and trying to apply their assignements to this competition.

CS231N: http://cs231n.github.io/assignments2018/assignment1/
<p>sci-kit knn: https://scikit-learn.org/stable/modules/neighbors.html

Main differences between assignment and this notebook:
    - we will be using sci-kit implementation of knn and not written by us
    

<h1>DATA LOADING

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.listdir('../../data')
assert 'out_breed.csv' in os.listdir('../../data') # this assert breaks if the data is configured uncorrectly

In [3]:
breeds = pd.read_csv('../../data/out_breed.csv')
colors = pd.read_csv('../../data/out_color.csv')
states = pd.read_csv('../../data/out_state.csv')
train  = pd.read_csv('../../data/out_train.csv')
test   = pd.read_csv('../../data/out_test.csv')
sub    = pd.read_csv('../../data/out_submission.csv')

<h1>MODEL</h1>

In [4]:
from knnModel import PredictiveModel

<h1>EXAMPLE USAGE</h1>

In [5]:
"""
this is a really primitive data cleaning to make KNN works: we drop the followings
- AdoptionSpeed, is target
- Unnamed:0, dataset_type, is useless
- Name, RescuerId, Description, PhotoAmt, VideoAmt, PetID: this are all strings valued not able to be processed by KNN
"""
non_numerical_cols = ["Unnamed: 0", "dataset_type", "Name", "RescuerID", "Description", "PhotoAmt","VideoAmt","PetID"]
X = train.drop(["AdoptionSpeed"] + non_numerical_cols, axis=1)
X_test = test.drop(non_numerical_cols, axis=1)

"""
Y is our target value, Adoption Speed can be a value [1,2,3,4]
"""
Y = train['AdoptionSpeed']

assert X.shape[0] == Y.shape[0]

In [6]:
model = PredictiveModel("example_usage_model")
model.train(X, Y)
predictions = model.predict(X_test)
assert len(predictions)

Sat Mar  2 00:39:37 2019 [example_usage_model.__init__] initialized succesfully


<h1>VALIDATION

In [7]:
dogs = train[train.Type == 1]
cats = train[train.Type == 2]

In [8]:
X = train.drop(["AdoptionSpeed", "Unnamed: 0", "dataset_type", "Name", "RescuerID", "Description", "PhotoAmt","VideoAmt","PetID"], axis=1)
"""
Y is our target value, Adoption Speed can be a value [1,2,3,4]
"""
Y = train['AdoptionSpeed']

assert X.shape[0] == Y.shape[0]

In [9]:
"""
Data Fields
PetID - Unique hash ID of pet profile
AdoptionSpeed - Categorical speed of adoption. Lower is faster. This is the value to predict. See below section for more info.
Type - Type of animal (1 = Dog, 2 = Cat)
Name - Name of pet (Empty if not named)
Age - Age of pet when listed, in months
Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)
Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
Color1 - Color 1 of pet (Refer to ColorLabels dictionary)
Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
Color3 - Color 3 of pet (Refer to ColorLabels dictionary)
MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
Quantity - Number of pets represented in profile
Fee - Adoption fee (0 = Free)
State - State location in Malaysia (Refer to StateLabels dictionary)
RescuerID - Unique hash ID of rescuer
VideoAmt - Total uploaded videos for this pet
PhotoAmt - Total uploaded photos for this pet
Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.
"""
pass

In [10]:
categorical_col = ["Type","Gender","Vaccinated","Dewormed","Sterilized","Breed1","Breed2","Color1","Color2","Color3","State"]
X.drop(categorical_col, axis=1).columns
#X = X.drop(categorical_col, axis=1)

Index(['Age', 'MaturitySize', 'FurLength', 'Health', 'Quantity', 'Fee'], dtype='object')

In [30]:
cols = X.columns
print(cols)

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State'],
      dtype='object')


In [13]:
17*17

289

In [17]:
def subsets(i):
    if i == -1:
        return [[]]
    else:
        res = []
        for subset in subsets(i - 1):
            res.append(subset + [cols[i]])
            res.append(subset)
        return res

In [58]:
def twosets():
    res = set([(col, '*') for col in cols])
    for i, col in enumerate(cols):
        for j, ccol in enumerate(cols):
            if i!=j:
                res.add(tuple(sorted([col, ccol])))
    return [list(e) for e in res]

In [59]:
selected_cols = twosets()

In [66]:
maxs = 0
besti = None
for i, csel in enumerate(selected_cols):
    if i % 30 == 0: print(i)
    x = X.drop([col for col in cols if col not in selected_cols[0]], axis=1)
    model = PredictiveModel("validation_model_"+str(csel[0])+"_"+csel[1])
    score = model.validation(x, Y, method=2)
    if score > maxs:
        maxs = score
        besti = i
        print(maxs)

0
Sat Mar  2 01:01:25 2019 [validation_model_Color1_Health.__init__] initialized succesfully
0.002410221051921213
Sat Mar  2 01:01:26 2019 [validation_model_State_Sterilized.__init__] initialized succesfully
Sat Mar  2 01:01:27 2019 [validation_model_Breed1_Sterilized.__init__] initialized succesfully
Sat Mar  2 01:01:29 2019 [validation_model_Breed1_Health.__init__] initialized succesfully
Sat Mar  2 01:01:30 2019 [validation_model_Fee_Vaccinated.__init__] initialized succesfully
Sat Mar  2 01:01:31 2019 [validation_model_Age_Fee.__init__] initialized succesfully
Sat Mar  2 01:01:32 2019 [validation_model_Color3_FurLength.__init__] initialized succesfully
Sat Mar  2 01:01:34 2019 [validation_model_Sterilized_Type.__init__] initialized succesfully
Sat Mar  2 01:01:35 2019 [validation_model_Age_MaturitySize.__init__] initialized succesfully
Sat Mar  2 01:01:36 2019 [validation_model_Breed2_Sterilized.__init__] initialized succesfully
Sat Mar  2 01:01:37 2019 [validation_model_Color2_Qua

Sat Mar  2 01:03:18 2019 [validation_model_Quantity_Sterilized.__init__] initialized succesfully
90
Sat Mar  2 01:03:20 2019 [validation_model_Health_*.__init__] initialized succesfully
Sat Mar  2 01:03:21 2019 [validation_model_Vaccinated_*.__init__] initialized succesfully
Sat Mar  2 01:03:22 2019 [validation_model_Breed1_MaturitySize.__init__] initialized succesfully
Sat Mar  2 01:03:23 2019 [validation_model_Health_State.__init__] initialized succesfully
Sat Mar  2 01:03:25 2019 [validation_model_Color1_Sterilized.__init__] initialized succesfully
Sat Mar  2 01:03:26 2019 [validation_model_Breed1_Color1.__init__] initialized succesfully
Sat Mar  2 01:03:27 2019 [validation_model_Quantity_*.__init__] initialized succesfully
Sat Mar  2 01:03:28 2019 [validation_model_Breed1_Quantity.__init__] initialized succesfully
Sat Mar  2 01:03:30 2019 [validation_model_Color1_*.__init__] initialized succesfully
Sat Mar  2 01:03:31 2019 [validation_model_Fee_FurLength.__init__] initialized succe

In [67]:
maxs

0.002410221051921213

In [79]:
model = PredictiveModel("validation_model_no_categorical")

Sat Mar  2 00:33:04 2019 [validation_model_no_categorical.__init__] initialized succesfully


In [80]:
X = X.reset_index(drop=True)
Y = Y.reset_index(drop=True)

In [81]:
model.validation(X, Y, method=1)

Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.validation] start validation method 1
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.train] start training
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.train] trained succefully
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.predict] start predictions
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.predict] predicted succesfully
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.evaluate] start evaluation
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.evaluate] evaluated succesfully
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.validation] single score = 0.14250968093848493 
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.train] start training
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.train] trained succefully
Sat Mar  2 00:33:06 2019 [validation_model_no_categorical.predict] start predictions
Sat Mar  2 00:33:07 2019 [validation

0.13190454691363668

In [53]:
model.validation(X, Y, method=2)

Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.validation] start validation method 2
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.train] start training
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.train] trained succefully
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.predict] start predictions
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.predict] predicted succesfully
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.evaluate] start evaluation
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.evaluate] evaluated succesfully
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.validation] single score = 0.10189602001345921 
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.train] start training
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.train] trained succefully
Sat Mar  2 00:29:33 2019 [validation_model_no_categorical_type.predict]

0.09920552975604653